# LSTM Models for Regression

In [84]:
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM

from bokeh.layouts import column
from bokeh.plotting import figure
from bokeh.io import output_notebook, show


np.random.seed(17)
output_notebook()

Loading BokehJS ...

### Design some artificial data

In [2]:
param = 20.
small_periods = 200
scale = 1.1
pick_freq = 10
dataset_1 = np.arange(param)
dataset_2 = np.arange(param)
dataset_3 = np.arange(param)

for i in range(1, small_periods):
    # Simple case
    d_1 = (np.arange(param) + (pick_freq * i  / 2)) * (scale + i / 50)
    dataset_1 = np.concatenate((dataset_1, d_1))
    # Interesting case
    d_2 = (np.arange(param) + (pick_freq * np.sin(i))) * (scale + i / 1)
    dataset_2 = np.concatenate((dataset_2, d_2))
    # Another interesting one
    d_3 = (np.arange(param) + (pick_freq * (i / 2) * np.sin(i))) * (scale + i / 100)
    dataset_3 = np.concatenate((dataset_3, d_3))


print(dataset_1.shape)
print(dataset_2.shape)
print(dataset_3.shape)

(4000,)
(4000,)
(4000,)


In [3]:
p_1 = figure(title='Dataset 1', plot_height=400, plot_width=900)
p_1.line(np.arange(len(dataset_1)), dataset_1, line_width=1)
p_2 = figure(title='Dataset 2', plot_height=400, plot_width=900)
p_2.line(np.arange(len(dataset_2)), dataset_2, line_width=1)
p_3 = figure(title='Dataset 3', plot_height=400, plot_width=900)
p_3.line(np.arange(len(dataset_3)), dataset_3, line_width=1)
show(column(p_1, p_2, p_3))

### Prepare data

In [4]:
scaler_1 = MinMaxScaler(feature_range=(0, 1))
scaler_2 = MinMaxScaler(feature_range=(0, 1))
scaler_3 = MinMaxScaler(feature_range=(0, 1))

ds_1 = scaler_1.fit_transform(dataset_1.reshape(-1, 1))
ds_2 = scaler_2.fit_transform(dataset_2.reshape(-1, 1))
ds_3 = scaler_3.fit_transform(dataset_3.reshape(-1, 1))

# scaler.inverse_transform(array) for inverse

In [ ]:
# Double check
p_1 = figure(title='Dataset 1 Scaled', plot_height=400, plot_width=900)
p_1.line(np.arange(len(ds_1)), ds_1.reshape(-1), line_width=1)
p_2 = figure(title='Dataset 2 Scaled', plot_height=400, plot_width=900)
p_2.line(np.arange(len(ds_2)), ds_2.reshape(-1), line_width=1)
p_3 = figure(title='Dataset 3 Scaled', plot_height=400, plot_width=900)
p_3.line(np.arange(len(ds_3)), ds_3.reshape(-1), line_width=1)
show(column(p_1, p_2, p_3))

In [5]:
def create_input(data, window_size=1):
    x, y = [], []
    for i in range(len(data) - window_size - 1):
        x.append(data[i:i + window_size, 0])
        y.append(data[i + window_size, 0])
    x, y = np.array(x), np.array(y)
    # Reshape input to [samples, timesteps, features]
    return x.reshape(x.shape[0], 1, x.shape[1]), y

def split_data(data, percentage=0.8):
    idx = int(len(data) * percentage)
    return data[:idx,:], data[idx:,:]

In [6]:
def get_plot_data(dataset, train, test, window_size):
    print('Incoming shapes: {}, {}, {}'.format(dataset.shape, train.shape, test.shape))
    train_plot = np.empty_like(dataset)
    train_plot[:, :] = np.nan
    train_plot[window_size:len(train)+window_size, :] = train
    # shift test predictions for plotting
    test_plot = np.empty_like(dataset)
    test_plot[:, :] = np.nan
    test_plot[len(train)+(window_size*2)+1:len(dataset)-1, :] = test
    x_axis = np.arange(len(dataset))
    print('Outcoming shapes: {}, {}, {}'.format(dataset.shape, train_plot.shape, test_plot.shape))
    return (
        (x_axis, dataset.reshape(-1)),
        (x_axis, train_plot.reshape(-1)),
        (x_axis, test_plot.reshape(-1)),
    )

def show_error(train, test):
    train_error = np.sqrt(mean_squared_error(train[0].reshape(-1), train[1].reshape(-1)))
    print('Train Score: %.2f RMSE' % (train_error))
    test_error = np.sqrt(mean_squared_error(test[0].reshape(-1), test[1].reshape(-1)))
    print('Test Score: %.2f RMSE' % (test_error))

def show_result(data, name):
    p = figure(title=name, plot_height=400, plot_width=900)
    p.line(data[0][0], data[0][1], line_width=1, color='blue', alpha=0.6)
    p.line(data[1][0], data[1][1], line_width=1, color='orange', alpha=0.6)
    p.line(data[2][0], data[2][1], line_width=2, color='green', alpha=0.6)
    show(p)

## Model 1 for Dataset 1 - Shallow LSTM

In [239]:
batch_size_1 = 16
window_size_1 = 1
timesteps_1 = 1

train_1, test_1 = split_data(ds_1)
train_1_x, train_1_y = create_input(train_1, window_size_1)
test_1_x, test_1_y = create_input(test_1, window_size_1)

print(
    """
    Train shape: {}, test shape: {}
    Train input shape: {}, train output shape {}
    Test input shape {}, test output shape {}
    """.format(train_1.shape, test_1.shape,
               train_1_x.shape, train_1_y.shape,
               test_1_x.shape, test_1_y.shape))


    Train shape: (3200, 1), test shape: (800, 1)
    Train input shape: (3198, 1, 1), train output shape (3198,)
    Test input shape (798, 1, 1), test output shape (798,)
    


In [243]:
model_1 = Sequential()
model_1.add(LSTM(96, return_sequences=True, input_shape=(1, window_size_1)))
# model_1.add(LSTM(64+128+64, return_sequences=True))
# model_1.add(LSTM(96, return_sequences=True))
model_1.add(LSTM(96))
model_1.add(Dense(1))
model_1.compile(
    loss='mse', 
    optimizer='adam',
)

history_1 = model_1.fit(
    train_1_x, train_1_y,
    batch_size=batch_size_1, epochs=15, shuffle=False,
)

Epoch 1/15
3198/3198 [==============================] - 15s 5ms/step - loss: 1.0294e-05
Epoch 2/15
3198/3198 [==============================] - 2s 525us/step - loss: 0.0064
Epoch 3/15
3198/3198 [==============================] - 2s 521us/step - loss: 0.0032
Epoch 4/15
3198/3198 [==============================] - 2s 504us/step - loss: 8.2671e-04
Epoch 5/15
3198/3198 [==============================] - 2s 508us/step - loss: 9.2699e-05
Epoch 6/15
3198/3198 [==============================] - 2s 508us/step - loss: 1.4080e-05
Epoch 7/15
3198/3198 [==============================] - 2s 529us/step - loss: 9.1097e-06
Epoch 8/15
3198/3198 [==============================] - 2s 531us/step - loss: 7.7313e-06
Epoch 9/15
3198/3198 [==============================] - 2s 549us/step - loss: 6.6992e-06
Epoch 10/15
3198/3198 [==============================] - 2s 548us/step - loss: 5.8809e-06
Epoch 11/15
3198/3198 [==============================] - 2s 522us/step - loss: 5.2224e-06
Epoch 12/15
3198/3198 [=====

In [244]:
train_predict_1 = model_1.predict(train_1_x)
test_predict_1 = model_1.predict(test_1_x)
print(train_predict_1.shape, test_predict_1.shape)

train_predict_1 = scaler_1.inverse_transform(train_predict_1)
test_predict_1 = scaler_1.inverse_transform(test_predict_1)
print(train_predict_1.shape, test_predict_1.shape)

train_1_y_unscaled = scaler_1.inverse_transform([train_1_y])
test_1_y_unscaled = scaler_1.inverse_transform([test_1_y])
print(train_1_y_unscaled.shape, test_1_y_unscaled.shape)

(3198, 1) (798, 1)
(3198, 1) (798, 1)
(1, 3198) (1, 798)


In [245]:
m_1 = figure(title='Learning curve for Model 1', plot_height=400, plot_width=900)
m_1.line(np.arange(len(history_1.history['loss'])), history_1.history['loss'], line_width=1)
show(m_1)

In [246]:
show_error((train_1_y_unscaled, train_predict_1), (test_1_y_unscaled, test_predict_1))
show_result(
    get_plot_data(dataset_1.reshape(-1,1), 
                  train_predict_1, 
                  test_predict_1, window_size_1),
    'Model 1')

Train Score: 23.42 RMSE
Test Score: 20.28 RMSE
Incoming shapes: (4000, 1), (3198, 1), (798, 1)
Outcoming shapes: (4000, 1), (4000, 1), (4000, 1)


Train Score: 12.88 RMSE
Test Score: 13.07 RMSE

Train Score: 12.63 RMSE
Test Score: 11.55 RMSE            64+128+32

Train Score: 18.91 RMSE
Test Score: 28.98 RMSE            64+128+64

## Model 2 for Dataset 2 - Stacked LSTM

In [88]:
batch_size_2 = 64
window_size_2 = 64
timesteps_2 = 1

train_2, test_2 = split_data(ds_2)
train_2_x, train_2_y = create_input(train_2, window_size_2)
test_2_x, test_2_y = create_input(test_2, window_size_2)

print(
    """
    Train shape: {}, test shape: {}
    Train input shape: {}, train output shape {}
    Test input shape {}, test output shape {}
    """.format(train_2.shape, test_2.shape,
               train_2_x.shape, train_2_y.shape,
               test_2_x.shape, test_2_y.shape))


    Train shape: (3200, 1), test shape: (800, 1)
    Train input shape: (3135, 1, 64), train output shape (3135,)
    Test input shape (735, 1, 64), test output shape (735,)
    


In [89]:
model_2 = Sequential()
model_2.add(LSTM(768, return_sequences=True, input_shape=(1, window_size_2)))
model_2.add(LSTM(768, return_sequences=True))
model_2.add(LSTM(768, return_sequences=True))
model_2.add(LSTM(768))
model_2.add(Dense(1))
model_2.compile(
    loss='mse', 
    optimizer='adam',
)

history_2 = model_2.fit(
    train_2_x, train_2_y,
    batch_size=batch_size_2, epochs=700, shuffle=False,
)

Epoch 1/700
3135/3135 [==============================] - 46s 15ms/step - loss: 0.0144
Epoch 2/700
3135/3135 [==============================] - 27s 8ms/step - loss: 0.0051
Epoch 3/700
3135/3135 [==============================] - 26s 8ms/step - loss: 0.0048
Epoch 4/700
3135/3135 [==============================] - 26s 8ms/step - loss: 0.0050
Epoch 5/700
3135/3135 [==============================] - 28s 9ms/step - loss: 0.0036
Epoch 6/700
3135/3135 [==============================] - 26s 8ms/step - loss: 0.0054
Epoch 7/700
3135/3135 [==============================] - 26s 8ms/step - loss: 0.0137
Epoch 8/700
3135/3135 [==============================] - 28s 9ms/step - loss: 0.0063
Epoch 9/700
3135/3135 [==============================] - 26s 8ms/step - loss: 0.0043
Epoch 10/700
3135/3135 [==============================] - 26s 8ms/step - loss: 0.0054
Epoch 11/700
3135/3135 [==============================] - 26s 8ms/step - loss: 0.0080
Epoch 12/700
3135/3135 [==============================] - 27s 

3135/3135 [==============================] - 26s 8ms/step - loss: 6.2985e-05
Epoch 95/700
3135/3135 [==============================] - 25s 8ms/step - loss: 6.3614e-05
Epoch 96/700
3135/3135 [==============================] - 27s 9ms/step - loss: 6.2093e-05
Epoch 97/700
3135/3135 [==============================] - 25s 8ms/step - loss: 6.7669e-05
Epoch 98/700
3135/3135 [==============================] - 26s 8ms/step - loss: 6.9602e-05
Epoch 99/700
3135/3135 [==============================] - 26s 8ms/step - loss: 5.2621e-05
Epoch 100/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.0241e-04
Epoch 101/700
3135/3135 [==============================] - 25s 8ms/step - loss: 9.6976e-05
Epoch 102/700
3135/3135 [==============================] - 25s 8ms/step - loss: 5.8138e-05
Epoch 103/700
3135/3135 [==============================] - 26s 8ms/step - loss: 6.6860e-05
Epoch 104/700
3135/3135 [==============================] - 26s 8ms/step - loss: 7.3117e-05
Epoch 105/700
3135

3135/3135 [==============================] - 26s 8ms/step - loss: 1.3384e-04
Epoch 185/700
3135/3135 [==============================] - 27s 9ms/step - loss: 1.7431e-04
Epoch 186/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.2267e-04
Epoch 187/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.5194e-04
Epoch 188/700
3135/3135 [==============================] - 26s 8ms/step - loss: 9.4989e-05
Epoch 189/700
3135/3135 [==============================] - 25s 8ms/step - loss: 5.9041e-05
Epoch 190/700
3135/3135 [==============================] - 26s 8ms/step - loss: 4.0521e-05
Epoch 191/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.1628e-05
Epoch 192/700
3135/3135 [==============================] - 26s 8ms/step - loss: 2.7546e-05
Epoch 193/700
3135/3135 [==============================] - 25s 8ms/step - loss: 2.5305e-05
Epoch 194/700
3135/3135 [==============================] - 25s 8ms/step - loss: 2.4735e-05
Epoch 195/700

3135/3135 [==============================] - 25s 8ms/step - loss: 1.4335e-05
Epoch 275/700
3135/3135 [==============================] - 25s 8ms/step - loss: 1.5108e-05
Epoch 276/700
3135/3135 [==============================] - 25s 8ms/step - loss: 1.5384e-05
Epoch 277/700
3135/3135 [==============================] - 25s 8ms/step - loss: 1.9798e-05
Epoch 278/700
3135/3135 [==============================] - 27s 9ms/step - loss: 4.2842e-05
Epoch 279/700
3135/3135 [==============================] - 25s 8ms/step - loss: 3.0979e-04
Epoch 280/700
3135/3135 [==============================] - 25s 8ms/step - loss: 8.9715e-05
Epoch 281/700
3135/3135 [==============================] - 25s 8ms/step - loss: 5.4857e-05
Epoch 282/700
3135/3135 [==============================] - 25s 8ms/step - loss: 2.4256e-05
Epoch 283/700
3135/3135 [==============================] - 25s 8ms/step - loss: 1.4824e-05
Epoch 284/700
3135/3135 [==============================] - 25s 8ms/step - loss: 1.4560e-05
Epoch 285/700

3135/3135 [==============================] - 26s 8ms/step - loss: 3.2206e-05
Epoch 365/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.2702e-05
Epoch 366/700
3135/3135 [==============================] - 26s 8ms/step - loss: 9.4460e-05
Epoch 367/700
3135/3135 [==============================] - 26s 8ms/step - loss: 2.0709e-05
Epoch 368/700
3135/3135 [==============================] - 28s 9ms/step - loss: 3.4651e-05
Epoch 369/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.4800e-05
Epoch 370/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.6796e-05
Epoch 371/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.5243e-05
Epoch 372/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.4448e-05
Epoch 373/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.4754e-05
Epoch 374/700
3135/3135 [==============================] - 27s 9ms/step - loss: 3.3992e-05
Epoch 375/700

3135/3135 [==============================] - 27s 9ms/step - loss: 1.7406e-05
Epoch 455/700
3135/3135 [==============================] - 25s 8ms/step - loss: 1.5727e-05
Epoch 456/700
3135/3135 [==============================] - 24s 8ms/step - loss: 1.4761e-05
Epoch 457/700
3135/3135 [==============================] - 27s 9ms/step - loss: 1.5425e-05
Epoch 458/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.7499e-05
Epoch 459/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.9815e-05
Epoch 460/700
3135/3135 [==============================] - 26s 8ms/step - loss: 2.0525e-05
Epoch 461/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.9125e-05
Epoch 462/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.6283e-05
Epoch 463/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.1015e-05
Epoch 464/700
3135/3135 [==============================] - 27s 8ms/step - loss: 1.5970e-04
Epoch 465/700

3135/3135 [==============================] - 25s 8ms/step - loss: 3.2191e-05
Epoch 545/700
3135/3135 [==============================] - 25s 8ms/step - loss: 1.8196e-05
Epoch 546/700
3135/3135 [==============================] - 26s 8ms/step - loss: 5.6062e-05
Epoch 547/700
3135/3135 [==============================] - 27s 9ms/step - loss: 1.5779e-05
Epoch 548/700
3135/3135 [==============================] - 25s 8ms/step - loss: 3.5322e-05
Epoch 549/700
3135/3135 [==============================] - 25s 8ms/step - loss: 2.9386e-05
Epoch 550/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.8470e-05
Epoch 551/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.0139e-05
Epoch 552/700
3135/3135 [==============================] - 25s 8ms/step - loss: 8.6188e-05
Epoch 553/700
3135/3135 [==============================] - 25s 8ms/step - loss: 4.2819e-05
Epoch 554/700
3135/3135 [==============================] - 26s 8ms/step - loss: 3.3054e-05
Epoch 555/700

3135/3135 [==============================] - 27s 9ms/step - loss: 1.9264e-05
Epoch 635/700
3135/3135 [==============================] - 26s 8ms/step - loss: 2.1326e-05
Epoch 636/700
3135/3135 [==============================] - 29s 9ms/step - loss: 1.6029e-05
Epoch 637/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.5536e-05
Epoch 638/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.4993e-05
Epoch 639/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.4189e-05
Epoch 640/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.2826e-05
Epoch 641/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.1049e-05
Epoch 642/700
3135/3135 [==============================] - 26s 8ms/step - loss: 1.0805e-05
Epoch 643/700
3135/3135 [==============================] - 25s 8ms/step - loss: 7.6278e-06
Epoch 644/700
3135/3135 [==============================] - 26s 8ms/step - loss: 2.1180e-05
Epoch 645/700

In [90]:
train_predict_2 = model_2.predict(train_2_x)
test_predict_2 = model_2.predict(test_2_x)
print(train_predict_2.shape, test_predict_2.shape)

train_predict_2 = scaler_2.inverse_transform(train_predict_2)
test_predict_2 = scaler_2.inverse_transform(test_predict_2)
print(train_predict_2.shape, test_predict_2.shape)

train_2_y_unscaled = scaler_2.inverse_transform([train_2_y])
test_2_y_unscaled = scaler_2.inverse_transform([test_2_y])
print(train_2_y_unscaled.shape, test_2_y_unscaled.shape)

(3135, 1) (735, 1)
(3135, 1) (735, 1)
(1, 3135) (1, 735)


In [91]:
m_2 = figure(title='Learning curve for Model 2', plot_height=400, plot_width=900)
m_2.line(np.arange(len(history_2.history['loss'])), history_2.history['loss'], line_width=1)
show(m_2)

In [92]:
show_error((train_2_y_unscaled, train_predict_2), (test_2_y_unscaled, test_predict_2))
show_result(
    get_plot_data(dataset_2.reshape(-1,1), 
                  train_predict_2, 
                  test_predict_2, window_size_2),
    'Model 2')

Train Score: 63.63 RMSE
Test Score: 46.26 RMSE
Incoming shapes: (4000, 1), (3135, 1), (735, 1)
Outcoming shapes: (4000, 1), (4000, 1), (4000, 1)


In [93]:
model_2.save('lstm_regression_model_2.h2')

### Model tunning log
For 256x3 B-32 W-1 E-50

Train Score: 448.38 RMSE

Test Score: 796.60 RMSE

--------
B-64

Train Score: 442.07 RMSE

Test Score: 796.38 RMSE

--------
E - 300

Train Score: 434.92 RMSE

Test Score: 798.18 RMSE

--------
256x4 E-50

Train Score: 447.34 RMSE

Test Score: 797.42 RMSE

--------
W-16

Train Score: 432.92 RMSE

Test Score: 780.39 RMSE

--------
W-64

Train Score: 350.90 RMSE

Test Score: 348.58 RMSE

--------
512x4 E-10

Train Score: 775.75 RMSE

Test Score: 418.81 RMSE

--------
E-1700

Train Score: 66.73 RMSE

Test Score: 103.31 RMSE

*Stopped trainning at the epoch 200*

--------
768x4 E-700

Train Score: 63.63 RMSE

Test Score: 46.26 RMSE

*Stopped trainning at the epoch 100*

---------


# Model 3 for Dataset 3 - Stacked LSTM

In [272]:
batch_size_3 = 64
window_size_3 = 1
timesteps_3 = 1

train_3, test_3 = split_data(ds_3)
train_3_x, train_3_y = create_input(train_3, window_size_3)
test_3_x, test_3_y = create_input(test_3, window_size_3)

print(
    """
    Train shape: {}, test shape: {}
    Train input shape: {}, train output shape {}
    Test input shape {}, test output shape {}
    """.format(train_3.shape, test_3.shape,
               train_3_x.shape, train_3_y.shape,
               test_3_x.shape, test_3_y.shape))


    Train shape: (3200, 1), test shape: (800, 1)
    Train input shape: (3198, 1, 1), train output shape (3198,)
    Test input shape (798, 1, 1), test output shape (798,)
    


In [273]:
model_3 = Sequential()
model_3.add(LSTM(1024, return_sequences=True, input_shape=(1, window_size_3)))
model_3.add(LSTM(1024, return_sequences=True))
model_3.add(LSTM(1024, return_sequences=True))
model_3.add(LSTM(1024))
model_3.add(Dense(1))
model_3.compile(
    loss='mse', 
    optimizer='adam',
)

history_3 = model_3.fit(
    train_3_x, train_3_y,
    batch_size=batch_size_3, epochs=7, shuffle=False,
)

Epoch 1/7
3198/3198 [==============================] - 72s 22ms/step - loss: 0.0345
Epoch 2/7
3198/3198 [==============================] - 48s 15ms/step - loss: 0.0014
Epoch 3/7
3198/3198 [==============================] - 46s 14ms/step - loss: 7.7808e-04
Epoch 4/7
3198/3198 [==============================] - 46s 14ms/step - loss: 7.8217e-04
Epoch 5/7
3198/3198 [==============================] - 52s 16ms/step - loss: 7.8312e-04
Epoch 6/7
3198/3198 [==============================] - 58s 18ms/step - loss: 7.8582e-04
Epoch 7/7
3198/3198 [==============================] - 47s 15ms/step - loss: 7.8568e-04


In [274]:
train_predict_3 = model_3.predict(train_3_x)
test_predict_3 = model_3.predict(test_3_x)
print(train_predict_3.shape, test_predict_3.shape)

train_predict_3 = scaler_3.inverse_transform(train_predict_3)
test_predict_3 = scaler_3.inverse_transform(test_predict_3)
print(train_predict_3.shape, test_predict_3.shape)

train_3_y_unscaled = scaler_3.inverse_transform([train_3_y])
test_3_y_unscaled = scaler_3.inverse_transform([test_3_y])
print(train_3_y_unscaled.shape, test_3_y_unscaled.shape)

(3198, 1) (798, 1)
(3198, 1) (798, 1)
(1, 3198) (1, 798)


In [275]:
m_3 = figure(title='Learning curve for Model 3', plot_height=400, plot_width=900)
m_3.line(np.arange(len(history_3.history['loss'])), history_3.history['loss'], line_width=1)
show(m_3)

In [276]:
show_error((train_3_y_unscaled, train_predict_3), (test_3_y_unscaled, test_predict_3))
show_result(
    get_plot_data(dataset_3.reshape(-1,1), 
                  train_predict_3, 
                  test_predict_3, window_size_3),
    'Model 2')

Train Score: 162.20 RMSE
Test Score: 399.25 RMSE
Incoming shapes: (4000, 1), (3198, 1), (798, 1)
Outcoming shapes: (4000, 1), (4000, 1), (4000, 1)


In [277]:
model_3.save('lstm_regression_model_3.h2')